In [1]:
import numpy as np
import symforce
symforce.set_epsilon_to_symbol()
from symforce.values import Values
from symforce.opt.factor import Factor
import symforce.symbolic as sf
from symforce.opt.optimizer import Optimizer

In [2]:
def meas_residual(
        pose: sf.V2, landmark: sf.V2, meas: sf.V2, epsilon: sf.Scalar, 
    ) -> sf.V1:

        Q = np.array([[0.0,0.0],[0.0,0.0]])
        rng_std = .5
        bearing_std = .5
        Q[0, 0] = rng_std**2
        Q[1, 1] = bearing_std**2

        Q_I = np.linalg.inv(Q)

        rng, bearing = meas
        # predicted measurement
        dm = sf.V2(landmark - pose)
        z_pred = sf.V2([[dm.norm(epsilon=epsilon)],[sf.atan2(dm[1], dm[0],epsilon=epsilon)]])

        # error
        e_z = meas - z_pred
        # cost
        J = e_z@Q_I@e_z.T

        return sf.V1(J)

In [3]:
def odometry_residual(
        pose_0: sf.V2, pose_1: sf.V2, odom: sf.V2, epsilon: sf.Scalar, 
    ) -> sf.V1:

        R = np.array([[0.0,0.0],[0.0,0.0]]) 
        odom_x_std = 3
        odom_y_std = 3
        R[0, 0] = odom_x_std**2
        R[1, 1] = odom_y_std**2
        R_I = np.linalg.inv(R)

        odom_pred = pose_1-pose_0    # error
        e_x = sf.V2(odom-odom_pred)
        # cost
        J = e_x@R_I@e_x.T
        return sf.V1(J)

In [11]:
def optimize(x,lm,odom,z):
    initial_values = Values(
        poses=[sf.V2(i,j) for i,j in x],
        landmarks=[sf.V2(i,j) for i,j in lm],
        odom=[sf.V2(i,j) for i,j in odom],
        meas=[sf.V2(i,j) for i,j,k,m in z],
        epsilon=sf.numeric_epsilon,
    )

    factors = []

    # Bearing factors

    for j in range(len(z)):
        factors.append(Factor(
            residual=meas_residual,
            keys=[f"poses[{int(z[j][2])}]", f"landmarks[{int(z[j][3])}]", f"meas[{j}]", "epsilon"],
        ))

    # Odometry factors
    for i in range(len(x) - 1):
        factors.append(Factor(
            residual=odometry_residual,
            keys=[f"poses[{i}]", f"poses[{i + 1}]", f"odom[{i}]", "epsilon"],
        ))

    params = Optimizer.Params(verbose=False)
    optimizer = Optimizer(
        factors=factors,
        optimized_keys=[f"poses[{i}]" for i in range(len(x))],
        params=params,
    )

    result = optimizer.optimize(initial_values)

    return result, factors

In [12]:
def update_init_values(initial_values, x, lm, odom, z):
    # initial_values: Previous initial value dictionary generated
    # x: Newest x,y coordinate from rover np.array([x, y]) that will be appended to old array
    # lm: Newest SET of landmarks np.array([[x1,y1],[x2,y2],...]) that will replace old landmarks
    # odom: Newest x,y odom from rover np.array([x, y]) that will be appended to old array
    # z: Newest range, bearing, pose, landmark np.array([]) that will be appended to old array
    
    x_new = np.vstack([np.array(initial_values['poses']), x])
    lm_new = lm
    odom_new = np.vstack([np.array(initial_values['odom']), odom])
    z_new = np.vstack([np.array(initial_values['meas']), z[:,0:2]])
    
    initial_values = Values(
        poses=[sf.V2(i,j) for i,j in x_new],
        landmarks=[sf.V2(i,j) for i,j in lm_new],
        odom=[sf.V2(i,j) for i,j in odom_new],
        meas=[sf.V2(i,j) for i,j in z_new],
        epsilon=sf.numeric_epsilon,
    )
    
    return initial_values

In [13]:
def update_factor_graph(factors, x, lm, odom, z):
    # Bearing factors

    for j in range(len(z)):
        factors.append(Factor(
            residual=meas_residual,
            keys=[f"poses[{int(z[j][2])}]", f"landmarks[{int(z[j][3])}]", f"meas[{j}]", "epsilon"],
        ))

    # Odometry factors
    for i in range(len(x) - 1):
        factors.append(Factor(
            residual=odometry_residual,
            keys=[f"poses[{i}]", f"poses[{i + 1}]", f"odom[{i}]", "epsilon"],
        ))

In [17]:
lm =np.array([
        [0, 2],
        [4, 6],
        [9, 1],
        [4,2],
        [9, 1.5],
        [9,10],
        [-5,15],
        [-7,15],])
x=np.array([[0,0],
             [1.98,.001]])
odom=np.array([[1.98,.001]])

# Maybe do range, bearing, pose, landmark
z = np.array([[2,1.57079633,0,0]])

initial_values = Values(
        poses=[sf.V2(i,j) for i,j in x],
        landmarks=[sf.V2(i,j) for i,j in lm],
        odom=[sf.V2(i,j) for i,j in odom],
        meas=[sf.V2(i,j) for i,j,k,m in z],
        epsilon=sf.numeric_epsilon,
    )
# initial_values['poses'] = []
type(initial_values['poses'])
initial_values['poses']
np.vstack([np.array(initial_values['meas']), [2.0, 2.0]])

optim, factors = optimize(x, lm, odom ,z)
optim

factors[0].name

'meas_residual'

In [22]:
xh = optim.optimized_values['poses']
lh = optim.optimized_values['landmarks']

print(type(optim.error()))

<class 'float'>


In [27]:
factors

z_num = 0
for lcv in range(len(factors)):
    if factors[lcv].name == 'meas_residual':
        z_num += 1

z_num

1

In [30]:
len(initial_values['poses'])

2

In [50]:
test_mat = sf.M(2,1)




type(test_mat)

symforce.geo.matrix.Matrix21